<center><h1>Swin Transformer</h1> </center>

<center><p><a href="http://arxiv.org/abs/2103.14030">Swin Transformer: Hierarchical Vision Transformer using Shifted Windows</a></p></center>

<img src="https://user-images.githubusercontent.com/24825165/121768619-038e6d80-cb9a-11eb-8cb7-daa827e7772b.png" width="1000"/>

[Code: https://github.com/microsoft/Swin-Transformer](https://github.com/microsoft/Swin-Transformer)


In [1]:
import collections.abc
from itertools import repeat

import torch
from torch import nn
from torch.nn.init import trunc_normal_

### Patch Embedding

In [2]:
class PatchEmbed(nn.Module):
    r""" Image to Patch Embedding.
    Args:
        img_size (int | tuple[int]): Image size. Default: 224
        patch_size (int | tuple[int]): Patch token size. Default: 4
        in_channels (int): Number of input image channels. Default: 3
        embed_dim (int): Number of linear projection output channels. Default: 96
    """

    def __init__(self, img_size=224, patch_size=4, in_channels=3, embed_dim=96):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution
        self.num_patches = patches_resolution[0] * patches_resolution[1]

        self.proj = nn.Conv2d(in_channels, embed_dim, patch_size, patch_size)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        """ Image to Patch Embedding.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, num_patches, embed_dim)
        """
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x

### Patch Merging

In [3]:
class PatchMerging(nn.Module):
    r""" Patch Merging Layer.
    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
    """

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = nn.LayerNorm(4 * dim)

    def forward(self, x):
        """ Patch Merging Layer.
        :param x: (batch_size, num_patches, dim)
        :return: (batch_size, num_patches/4, dim*2)
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # (B, H/2, W/2, C)
        x1 = x[:, 1::2, 0::2, :]  # (B, H/2, W/2, C)
        x2 = x[:, 0::2, 1::2, :]  # (B, H/2, W/2, C)
        x3 = x[:, 1::2, 1::2, :]  # (B, H/2, W/2, C)
        x = torch.cat([x0, x1, x2, x3], -1)  # (B, H/2, W/2, 4*C)
        x = x.view(B, -1, 4 * C)  # (B, H/2*W/2, 4*C)

        x = self.norm(x)
        x = self.reduction(x)  # (B, H/2*W/2, 2*C)

        return x

## Sublayers

### MLP

In [4]:
class Mlp(nn.Module):
    r""" Multilayer Perceptron.
    Args:
        in_features (int): Input dimension.
        hidden_features (int): Hidden dimension. Default: in_features if None
        out_features (int): Output dimension. Default: in_features if None
        drop_rate (float): Dropout rate. Default: 0
    """

    def __init__(self, in_features, hidden_features=None, out_features=None, drop_rate=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop_rate)

    def forward(self, x):
        """ Multilayer Perceptron.
        :param x: (batch_size, num_patches, in_features)
        :return: (batch_size, num_patches, out_features)
        """
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

### Window Based Multi-Head Self Attention (W-MSA)

In [5]:
class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        attn_drop_rate (float): Dropout ratio of attention weight. Default: 0
        proj_drop_rate (float): Dropout ratio of output. Default: 0
    """

    def __init__(self, dim, window_size, num_heads, attn_drop_rate=0., proj_drop_rate=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))

        relative_position_index = self._create_relative_position()
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3)
        self.attn_drop = nn.Dropout(attn_drop_rate)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_rate)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """ Window based multi-head self attention (W-MSA) module.
        :param x: (batch_size*num_windows, window_height*window_width, dim)
        :param mask: (num_windows, window_height*window_width). Default: None
        :return: (batch_size*num_windows, window_height*window_width, dim)
        """
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = (
            self.relative_position_bias_table[self.relative_position_index.view(-1)]
            .view(self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1))
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)

        attn = self.softmax(attn)
        attn = self.attn_drop(attn)

        x = attn @ v
        x = x.transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def _create_relative_position(self):
        """ Pair-wise relative position index for each token inside the window.
        :return: (window_height*window_width, window_height*window_width)
        """
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        # (Wh, Ww) x 2 -> (2, Wh, Ww)
        coords = torch.stack(torch.meshgrid([coords_h, coords_w], indexing="ij"))
        coords_flatten = torch.flatten(coords, 1)
        # [[[ 0, 0,-1,-1],[ 0, 0,-1,-1],[ 1, 1, 0, 0],[ 1, 1, 0, 0]],
        #  [[ 0,-1, 0,-1],[ 1, 0, 1, 0],[ 0,-1, 0,-1],[ 1, 0, 1, 0]]]
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        # [[[ 0, 0],[ 0,-1],[-1, 0],[-1,-1]],
        #   [ 0, 1],[ 0, 0],[-1, 1],[-1, 0]],
        #   [ 1, 0],[ 1,-1],[ 0, 0],[ 0,-1]],
        #   [ 1, 1],[ 1, 0],[ 0, 1],[ 0, 0]]]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()
        # [[[ 3, 1],[ 3, 0],[ 0, 1],[ 0, 0]],
        #   [ 3, 2],[ 3, 1],[ 0, 2],[ 0, 1]],
        #   [ 6, 1],[ 6, 0],[ 3, 1],[ 3, 0]],
        #   [ 6, 2],[ 6, 1],[ 3, 2],[ 3, 2]]]
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        # [[ 4, 3, 1, 0],
        #  [ 5, 4, 2, 1],
        #  [ 7, 6, 4, 3],
        #  [ 8, 7, 5, 4]]
        relative_position_index = relative_coords.sum(-1)
        return relative_position_index

## Swin-Transformer Block

In [6]:
class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        num_heads (int): Number of attention heads.
        window_size (int): Window size. Default: 7
        shift_size (int): Shift size for SW-MSA. Default: 0
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0, mlp_ratio=4.,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(
            dim=dim,
            window_size=to_2tuple(self.window_size),
            num_heads=num_heads,
            attn_drop_rate=attn_drop_rate,
            proj_drop_rate=drop_rate,
        )

        self.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0. else nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = Mlp(
            in_features=dim,
            hidden_features=int(dim * mlp_ratio),
            drop_rate=drop_rate,
        )

        attn_mask = self._create_mask()
        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        """ Swin Transformer Block.
        :param x: (batch_size, num_patches, dim)
        :return: (batch_size, num_patches, dim)
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2)) \
            if self.shift_size > 0 else x

        # window partition
        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)

        # reverse cyclic shift
        x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2)) \
            if self.shift_size > 0 else x

        x = x.view(B, H * W, C)
        x = shortcut + self.drop_path(x)

        # FFN
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def _create_mask(self):
        """ Calculate attention mask for SW-MSA.
        https://github.com/microsoft/Swin-Transformer/issues/38
        :return: (batch_size*num_windows, window_size*window_size, window_size*window_size)
        """
        if self.shift_size == 0:
            return None
        H, W = self.input_resolution
        img_mask = torch.zeros((1, H, W, 1))
        h_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        w_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        cnt = 0
        for h in h_slices:
            for w in w_slices:
                img_mask[:, h, w, :] = cnt
                cnt += 1

        # (batch_size*num_windows, window_size, window_size, 1)
        mask_windows = window_partition(img_mask, self.window_size)
        mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
        # (batch_size*num_windows, window_size*window_size, window_size*window_size)
        attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
        # mask those which are not 0 (attention between different regions)
        attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))

        return attn_mask

## Basic Layer

Swim Transformer Block + Patch Merging/None.

In [7]:
class BasicLayer(nn.Module):
    r""" A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        drop (float): Dropout rate. Default: 0
        attn_drop (float): Attention dropout rate. Default: 0
        drop_path_rate (float | tuple[float]): Stochastic depth rate. Default: 0
        downsample (PatchMerging | None): Downsample layer at the end of the layer. Default: None
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size, mlp_ratio=4.,
                 drop=0., attn_drop=0., drop_path_rate=0., downsample=None):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth

        self.blocks = nn.ModuleList([
            SwinTransformerBlock(
                dim=dim,
                input_resolution=input_resolution,
                num_heads=num_heads,
                window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2,
                mlp_ratio=mlp_ratio,
                drop_rate=drop,
                attn_drop_rate=attn_drop,
                drop_path_rate=drop_path_rate[i] if isinstance(drop_path_rate, list) else drop_path_rate,
            ) for i in range(depth)
        ])

        self.downsample = downsample(input_resolution, dim=dim) if downsample is not None else nn.Identity()

    def forward(self, x):
        """ A basic Swin Transformer layer for one stage.
        :param x: (batch_size, num_patches, dim)
        :return: (batch_size, num_patches/4, dim*2)
        """
        for blk in self.blocks:
            x = blk(x)
        x = self.downsample(x)
        return x

## Swin Transformer

In [8]:
class SwinTransformer(nn.Module):
    r""" Swin Transformer.
    Args:
        img_size (int | tuple[int]): Input image size. Default 224
        patch_size (int | tuple[int]): Patch size. Default: 4
        in_channels (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        embed_dim (int): Patch embedding dimension. Default: 96
        depths (tuple(int)): Depth of each Swin Transformer layer.
        num_heads (tuple(int)): Number of attention heads in different layers.
        window_size (int): Window size. Default: 7
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0.1
    """

    def __init__(self, img_size=224, patch_size=4, in_channels=3, num_classes=1000, embed_dim=96,
                 depths=(2, 2, 6, 2), num_heads=(3, 6, 12, 24), window_size=7, mlp_ratio=4.,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1):
        super().__init__()

        self.num_classes = num_classes
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))
        self.mlp_ratio = mlp_ratio

        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embed_dim=embed_dim,
        )
        patches_resolution = self.patch_embed.patches_resolution
        self.patches_resolution = patches_resolution

        self.pos_drop = nn.Dropout(drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(
                dim=int(embed_dim * 2 ** i_layer),
                input_resolution=(
                    patches_resolution[0] // (2 ** i_layer),
                    patches_resolution[1] // (2 ** i_layer)),
                depth=depths[i_layer],
                num_heads=num_heads[i_layer],
                window_size=window_size,
                mlp_ratio=self.mlp_ratio,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path_rate=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])],
                downsample=PatchMerging if (i_layer < self.num_layers - 1) else None,
            )
            self.layers.append(layer)

        self.norm = nn.LayerNorm(self.num_features)
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

        self.apply(_init_weights)

    def forward_features(self, x):
        """ Swin Transformer.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, in_channels)
        """
        x = self.patch_embed(x)  # (B, L, C)
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)  # (B, L, C)
        x = self.avg_pool(x.transpose(1, 2))  # (B, C, 1)
        x = torch.flatten(x, 1)  # (B, C)
        return x

    def forward(self, x):
        """ Swin Transformer.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, num_classes)
        """
        x = self.forward_features(x)
        x = self.head(x)
        return x

# Utils

## Window Partition

In [9]:
def window_partition(x, window_size):
    """ Feature maps -> windows.
    :param x: (batch_size, height, width, channels)
    :param window_size: window size.
    :return: (batch_size*num_windows, window_size, window_size, channels)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """ Windows -> feature maps.
    :param windows: (num_windows*batch_size, window_size, window_size, channels)
    :param window_size: window size.
    :param H: height of image.
    :param W: width of image.
    :return: (batch_size, height, width, channels)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

## Init Weight

In [10]:
def _init_weights(m):
    if isinstance(m, nn.Linear):
        trunc_normal_(m.weight, std=.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
        nn.init.constant_(m.bias, 0)
        nn.init.constant_(m.weight, 1.0)

## Drop Paths

In [11]:
class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample when applied
    in main path of residual blocks.
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample when applied
    in main path of residual blocks.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    # work with diff dim tensors, not just 2D ConvNets
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output

## _ntuple

In [12]:
# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return tuple(x)
        return tuple(repeat(x, n))

    return parse


to_2tuple = _ntuple(2)

# Summary

## Data

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

in_channel = 3
image_size = 224
image_size_384 = 384
batch_size = 2
num_classes = 1000
num_classes_in21k = 21843

data = torch.randn((batch_size, in_channel, image_size, image_size)).to(device)
data_384 = torch.randn((batch_size, in_channel, image_size_384, image_size_384)).to(device)

## Swin-T

In [14]:
from torchkeras import summary

net = SwinTransformer(
    img_size=224,
    patch_size=4,
    embed_dim=96,
    depths=(2, 2, 6, 2),
    num_heads=(3, 6, 12, 24),
    window_size=7,
    num_classes=num_classes,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                            [-1, 96, 56, 56]                4,704
LayerNorm-2                           [-1, 3136, 96]                  192
Dropout-3                             [-1, 3136, 96]                    0
LayerNorm-4                           [-1, 3136, 96]                  192
Linear-5                               [-1, 49, 288]               27,936
Softmax-6                            [-1, 3, 49, 49]                    0
Dropout-7                            [-1, 3, 49, 49]                    0
Linear-8                                [-1, 49, 96]                9,312
Dropout-9                               [-1, 49, 96]                    0
Identity-10                           [-1, 3136, 96]                    0
LayerNorm-11                          [-1, 3136, 96]                  192
Linear-12                            

## Swin-S

In [15]:
net = SwinTransformer(
    img_size=224,
    patch_size=4,
    embed_dim=96,
    depths=(2, 2, 18, 2),
    num_heads=(3, 6, 12, 24),
    window_size=7,
    num_classes=num_classes,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                            [-1, 96, 56, 56]                4,704
LayerNorm-2                           [-1, 3136, 96]                  192
Dropout-3                             [-1, 3136, 96]                    0
LayerNorm-4                           [-1, 3136, 96]                  192
Linear-5                               [-1, 49, 288]               27,936
Softmax-6                            [-1, 3, 49, 49]                    0
Dropout-7                            [-1, 3, 49, 49]                    0
Linear-8                                [-1, 49, 96]                9,312
Dropout-9                               [-1, 49, 96]                    0
Identity-10                           [-1, 3136, 96]                    0
LayerNorm-11                          [-1, 3136, 96]                  192
Linear-12                            

## Swin-B

In [16]:
net = SwinTransformer(
    img_size=224,
    patch_size=4,
    embed_dim=128,
    depths=(2, 2, 18, 2),
    num_heads=(4, 8, 16, 32),
    window_size=7,
    num_classes=num_classes,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                           [-1, 128, 56, 56]                6,272
LayerNorm-2                          [-1, 3136, 128]                  256
Dropout-3                            [-1, 3136, 128]                    0
LayerNorm-4                          [-1, 3136, 128]                  256
Linear-5                               [-1, 49, 384]               49,536
Softmax-6                            [-1, 4, 49, 49]                    0
Dropout-7                            [-1, 4, 49, 49]                    0
Linear-8                               [-1, 49, 128]               16,512
Dropout-9                              [-1, 49, 128]                    0
Identity-10                          [-1, 3136, 128]                    0
LayerNorm-11                         [-1, 3136, 128]                  256
Linear-12                            

## Swin-B 384x384

In [17]:
net = SwinTransformer(
    img_size=384,
    patch_size=4,
    embed_dim=128,
    depths=(2, 2, 18, 2),
    num_heads=(4, 8, 16, 32),
    window_size=12,
    num_classes=num_classes,
).to(device)

summary(net, input_data=data_384)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                           [-1, 128, 96, 96]                6,272
LayerNorm-2                          [-1, 9216, 128]                  256
Dropout-3                            [-1, 9216, 128]                    0
LayerNorm-4                          [-1, 9216, 128]                  256
Linear-5                              [-1, 144, 384]               49,536
Softmax-6                          [-1, 4, 144, 144]                    0
Dropout-7                          [-1, 4, 144, 144]                    0
Linear-8                              [-1, 144, 128]               16,512
Dropout-9                             [-1, 144, 128]                    0
Identity-10                          [-1, 9216, 128]                    0
LayerNorm-11                         [-1, 9216, 128]                  256
Linear-12                            

## Swin-L ImageNet-22k

In [18]:
net = SwinTransformer(
    img_size=224,
    patch_size=4,
    embed_dim=192,
    depths=(2, 2, 18, 2),
    num_heads=(6, 12, 24, 48),
    window_size=7,
    num_classes=num_classes_in21k,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                           [-1, 192, 56, 56]                9,408
LayerNorm-2                          [-1, 3136, 192]                  384
Dropout-3                            [-1, 3136, 192]                    0
LayerNorm-4                          [-1, 3136, 192]                  384
Linear-5                               [-1, 49, 576]              111,168
Softmax-6                            [-1, 6, 49, 49]                    0
Dropout-7                            [-1, 6, 49, 49]                    0
Linear-8                               [-1, 49, 192]               37,056
Dropout-9                              [-1, 49, 192]                    0
Identity-10                          [-1, 3136, 192]                    0
LayerNorm-11                         [-1, 3136, 192]                  384
Linear-12                            

## Swin-L ImageNet-22k 384x384

In [19]:
net = SwinTransformer(
    img_size=384,
    patch_size=4,
    embed_dim=192,
    depths=(2, 2, 18, 2),
    num_heads=(6, 12, 24, 48),
    window_size=12,
    num_classes=num_classes_in21k,
).to(device)

summary(net, input_data=data_384)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                           [-1, 192, 96, 96]                9,408
LayerNorm-2                          [-1, 9216, 192]                  384
Dropout-3                            [-1, 9216, 192]                    0
LayerNorm-4                          [-1, 9216, 192]                  384
Linear-5                              [-1, 144, 576]              111,168
Softmax-6                          [-1, 6, 144, 144]                    0
Dropout-7                          [-1, 6, 144, 144]                    0
Linear-8                              [-1, 144, 192]               37,056
Dropout-9                             [-1, 144, 192]                    0
Identity-10                          [-1, 9216, 192]                    0
LayerNorm-11                         [-1, 9216, 192]                  384
Linear-12                            